In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import pickle
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
import orjson

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
with open('dataset_train_files.pkl', 'rb') as fp:
    df_train = pickle.load(fp)
df_train.head()

,filename,kind,defect,size,class_name,class
0,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
1,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
2,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
5,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
6,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [4]:
gen = ImageDataGenerator()
gen

In [5]:
flow_test  = gen.flow_from_dataframe(df_test, y_col='class_name')
flow_train  = gen.flow_from_dataframe(df_train, y_col='class_name')
flow_test, flow_train

Found 360 validated image filenames belonging to 4 classes.
Found 840 validated image filenames belonging to 4 classes.


(<keras.preprocessing.image.DataFrameIterator at 0x13b7a8f82c8>,
 <keras.preprocessing.image.DataFrameIterator at 0x13b7a854fc8>)

In [6]:
df_test[df_test['class'] != flow_test.classes]

,filename,kind,defect,size,class_name,class


In [7]:
df_train[df_train['class'] != flow_train.classes]

,filename,kind,defect,size,class_name,class


In [8]:
?Dropout

Init signature: Dropout(rate, noise_shape=None, seed=None, **kwargs)
Docstring:     
Applies Dropout to the input.

Dropout consists in randomly setting
a fraction `rate` of input units to 0 at each update during training time,
which helps prevent overfitting.

# Arguments
    rate: float between 0 and 1. Fraction of the input units to drop.
    noise_shape: 1D integer tensor representing the shape of the
        binary dropout mask that will be multiplied with the input.
        For instance, if your inputs have shape
        `(batch_size, timesteps, features)` and
        you want the dropout mask to be the same for all timesteps,
        you can use `noise_shape=(batch_size, 1, features)`.
    seed: A Python integer to use as random seed.

# References
    - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](
       http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)
File:           c:\users\eros\venv37\lib\site-packages\keras\layers\core.py
Type:  

In [9]:
model = keras.Sequential()

model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(256, 256, 3))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [10]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [11]:
with open('model_raw_dropout.json', 'w') as fp:
    fp.write(model.to_json())

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)       

In [13]:
history = model.fit_generator(flow_train, epochs=100, verbose=2, validation_data=flow_test, callbacks=[
    ModelCheckpoint(
        'model_raw_dropout.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_raw_dropout')
])

Epoch 1/100


C:\Users\Eros\venv37\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.101196). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


 - 3s - loss: 4.3296 - accuracy: 0.3060 - val_loss: 1.3010 - val_accuracy: 0.3250
Epoch 2/100
 - 2s - loss: 1.3318 - accuracy: 0.3429 - val_loss: 1.4257 - val_accuracy: 0.3194
Epoch 3/100
 - 3s - loss: 1.3042 - accuracy: 0.4012 - val_loss: 1.1486 - val_accuracy: 0.4139
Epoch 4/100
 - 3s - loss: 1.1993 - accuracy: 0.4655 - val_loss: 1.1204 - val_accuracy: 0.5028
Epoch 5/100
 - 2s - loss: 1.1143 - accuracy: 0.5167 - val_loss: 1.2200 - val_accuracy: 0.4278
Epoch 6/100
 - 3s - loss: 1.0706 - accuracy: 0.5214 - val_loss: 1.4407 - val_accuracy: 0.5667
Epoch 7/100
 - 3s - loss: 1.0287 - accuracy: 0.5536 - val_loss: 1.7370 - val_accuracy: 0.5333
Epoch 8/100
 - 2s - loss: 0.9389 - accuracy: 0.6071 - val_loss: 1.7422 - val_accuracy: 0.5417
Epoch 9/100
 - 3s - loss: 0.9890 - accuracy: 0.5881 - val_loss: 1.6014 - val_accuracy: 0.5556
Epoch 10/100
 - 3s - loss: 0.9065 - accuracy: 0.6298 - val_loss: 0.9841 - val_accuracy: 0.5750
Epoch 11/100
 - 3s - loss: 0.9135 - accuracy: 0.6393 - val_loss: 1.1161

In [14]:
with open('train_raw_history.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history.history.items()}))